In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Hubert_large/train_21.npz')
test_data = np.load('./Hubert_large/test_21.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-09-16 21:38:06.410853: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-16 21:38:07.234384: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-09-16 21:38:09.560090: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-09-16 21:38:09.560175: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: mojo
2024-09-16 21:38:09.560181: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: mojo
2024-09-16 21:38:09.560337: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 535.183.1
2024-09-16 21:38:09.560366: I external/local_xla/xla/stream_executor/

22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 470ms/step - accuracy: 0.5565 - loss: 3.2033 - val_accuracy: 0.5407 - val_loss: 0.6928
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 467ms/step - accuracy: 0.6047 - loss: 0.6652 - val_accuracy: 0.6977 - val_loss: 0.6131
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 465ms/step - accuracy: 0.7352 - loss: 0.5479 - val_accuracy: 0.7500 - val_loss: 0.5394
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 471ms/step - accuracy: 0.7778 - loss: 0.4723 - val_accuracy: 0.7907 - val_loss: 0.4331
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 470ms/step - accuracy: 0.8471 - loss: 0.3542 - val_accuracy: 0.8547 - val_loss: 0.3486
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 472ms/step - accuracy: 0.8687 - loss: 0.2928 - val_accuracy: 0.8198 - val_loss: 0.3818
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 475ms/step - accuracy: 0.9121 - loss: 0.2285 - val_accuracy: 0.8256 - val_loss: 0.3516
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 471ms/step - accuracy: 0.8705 - loss: 0.2647 - val_accuracy: 0.848

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4989 - loss: 1.5645 - val_accuracy: 0.6163 - val_loss: 0.6571
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5409 - loss: 1.1486 - val_accuracy: 0.5581 - val_loss: 0.8518
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5907 - loss: 1.0091 - val_accuracy: 0.6047 - val_loss: 0.6512
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5952 - loss: 0.8687 - val_accuracy: 0.5640 - val_loss: 0.6497
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6064 - loss: 0.8025 - val_accuracy: 0.5698 - val_loss: 0.6499
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5746 - loss: 0.7633 - val_accuracy: 0.6453 - val_loss: 0.6094
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6195 - loss: 0.7088 - val_accuracy: 0.7151 - val_loss: 0.6016
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6758 - loss: 0.6750 - val_accuracy: 0.7151 - val_loss: 0.5870
Ep

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Hubert_large/train_22.npz')
test_data = np.load('./Hubert_large/test_22.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 468ms/step - accuracy: 0.5273 - loss: 1.8573 - val_accuracy: 0.4186 - val_loss: 0.7059
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 464ms/step - accuracy: 0.5076 - loss: 0.6904 - val_accuracy: 0.4360 - val_loss: 0.7614
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 461ms/step - accuracy: 0.5864 - loss: 0.6776 - val_accuracy: 0.6221 - val_loss: 0.6576
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 466ms/step - accuracy: 0.6464 - loss: 0.6229 - val_accuracy: 0.7616 - val_loss: 0.5267
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 471ms/step - accuracy: 0.7614 - loss: 0.5094 - val_accuracy: 0.7558 - val_loss: 0.5144
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 463ms/step - accuracy: 0.8338 - loss: 0.4230 - val_accuracy: 0.8488 - val_loss: 0.3639
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 461ms/step - accuracy: 0.8475 - loss: 0.3321 - val_accuracy: 0.8314 - val_loss: 0.3859
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 458ms/step - accuracy: 0.8893 - loss: 0.3035 - val_accuracy: 0.860

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5200 - loss: 1.4618 - val_accuracy: 0.4884 - val_loss: 1.1588
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5264 - loss: 1.5101 - val_accuracy: 0.5523 - val_loss: 0.6958
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5875 - loss: 1.0283 - val_accuracy: 0.5523 - val_loss: 0.6511
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5741 - loss: 0.9512 - val_accuracy: 0.6570 - val_loss: 0.6368
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6135 - loss: 0.8213 - val_accuracy: 0.5872 - val_loss: 0.6713
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5784 - loss: 0.8451 - val_accuracy: 0.5698 - val_loss: 0.6722
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6478 - loss: 0.7619 - val_accuracy: 0.7791 - val_loss: 0.5929
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6403 - loss: 0.7111 - val_accuracy: 0.7384 - val_loss: 0.6023
Epo

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Hubert_large/train_23.npz')
test_data = np.load('./Hubert_large/test_23.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 466ms/step - accuracy: 0.5225 - loss: 2.6880 - val_accuracy: 0.4186 - val_loss: 0.7055
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 461ms/step - accuracy: 0.5679 - loss: 0.6836 - val_accuracy: 0.4186 - val_loss: 0.7215
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 461ms/step - accuracy: 0.5807 - loss: 0.6779 - val_accuracy: 0.5058 - val_loss: 0.6890
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 458ms/step - accuracy: 0.6512 - loss: 0.6570 - val_accuracy: 0.6512 - val_loss: 0.6285
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.6424 - loss: 0.6402 - val_accuracy: 0.7151 - val_loss: 0.5818
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 457ms/step - accuracy: 0.7520 - loss: 0.5183 - val_accuracy: 0.7674 - val_loss: 0.5071
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 456ms/step - accuracy: 0.7730 - loss: 0.5002 - val_accuracy: 0.7558 - val_loss: 0.4889
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 10s 454ms/step - accuracy: 0.8472 - loss: 0.3994 - val_accuracy: 0.790

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5131 - loss: 1.4778 - val_accuracy: 0.5930 - val_loss: 0.8727
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4968 - loss: 1.5950 - val_accuracy: 0.6047 - val_loss: 0.7631
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5281 - loss: 1.2376 - val_accuracy: 0.4826 - val_loss: 0.7862
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5453 - loss: 1.0696 - val_accuracy: 0.4535 - val_loss: 0.7393
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5724 - loss: 1.0052 - val_accuracy: 0.5174 - val_loss: 0.7117
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5749 - loss: 0.8258 - val_accuracy: 0.5640 - val_loss: 0.6809
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5831 - loss: 0.8186 - val_accuracy: 0.6977 - val_loss: 0.6453
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5823 - loss: 0.7639 - val_accuracy: 0.5988 - val_loss: 0.6560
Epo

In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./Hubert_large/train_24.npz')
test_data = np.load('./Hubert_large/test_24.npz')

train_features = train_data['features']  
test_features = test_data['features']

train_labels = pd.read_csv('y_train.csv').Gender_bin
test_labels = pd.read_csv('y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# CNN model (using 1024 feature length)
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the CNN
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(test_features_cnn, test_labels_encoded)
print(f"CNN Test Accuracy: {cnn_test_accuracy:.4f}")

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# PyTorch CNN Model
class CNN1DModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN1DModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Prepare tensors for PyTorch
X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize PyTorch model, loss, and optimizer
input_channels = 1
num_classes = len(label_encoder.classes_)
model = CNN1DModel(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# PyTorch Training Loop
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# Test PyTorch model
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Evaluate PyTorch model
test_loss, test_accuracy = test_model(model, test_loader, criterion)
print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


Epoch 1/50


/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 485ms/step - accuracy: 0.4741 - loss: 2.5387 - val_accuracy: 0.4186 - val_loss: 0.6996
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 481ms/step - accuracy: 0.5410 - loss: 0.6899 - val_accuracy: 0.5814 - val_loss: 0.6916
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 480ms/step - accuracy: 0.4481 - loss: 0.6937 - val_accuracy: 0.4186 - val_loss: 0.6935
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 494ms/step - accuracy: 0.5215 - loss: 0.6927 - val_accuracy: 0.4186 - val_loss: 0.6958
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 493ms/step - accuracy: 0.5254 - loss: 0.6924 - val_accuracy: 0.4186 - val_loss: 0.6973
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 497ms/step - accuracy: 0.5394 - loss: 0.6912 - val_accuracy: 0.4186 - val_loss: 0.6988
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 491ms/step - accuracy: 0.5307 - loss: 0.6917 - val_accuracy: 0.4186 - val_loss: 0.7003
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.6344 - loss: 0.6833
CNN Test Accuracy: 0.5659
SVM Accura

/home/mojo/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4860 - loss: 1.5044 - val_accuracy: 0.4244 - val_loss: 1.0217
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5858 - loss: 1.3605 - val_accuracy: 0.5698 - val_loss: 0.6662
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5200 - loss: 1.2550 - val_accuracy: 0.6047 - val_loss: 0.6835
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5710 - loss: 1.0181 - val_accuracy: 0.5756 - val_loss: 0.6869
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5644 - loss: 0.9064 - val_accuracy: 0.5116 - val_loss: 0.7115
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5409 - loss: 0.8852 - val_accuracy: 0.5698 - val_loss: 0.6817
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5373 - loss: 0.8314 - val_accuracy: 0.5640 - val_loss: 0.6778
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6247 - loss: 0.6818 
ANN Test Accuracy: 0.6047
Epoch 1: Loss = 13.5891
Epoch 